In [2]:
from src.helpers import load_df_tweets, get_mentioned_coins
from src.paths import LOCAL_PROCESSED_DATA_PATH

from tqdm._tqdm_notebook import tqdm_notebook
import numpy as np

df = load_df_tweets()
df['mentioned_coins'] = df.words.progress_apply(get_mentioned_coins)
df['mentioned_coins'].explode().value_counts()

c:\users\muril\.virtualenvs\crypto-prophet-ckjwbuny\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  
100%|████████████████████████████████████████████████████████████████████████████| 69867/69867 [53:14<00:00, 21.87it/s]


BTC     4097
ETH      999
LTC      731
CAG      284
TRX      254
        ... 
ACRE       1
DENT       1
SYS        1
BIS        1
HAK        1
Name: mentioned_coins, Length: 786, dtype: int64

In [3]:
df.explode('mentioned_coins').groupby(['mentioned_coins','user_screen_name']).size().sort_values(ascending=False).head(20)

mentioned_coins  user_screen_name
BTC              rogerkver           1222
LTC              SatoshiLite          714
BTC              BarrySilbert         496
                 APompliano           428
                 ErikVoorhees         325
ETH              VitalikButerin       277
BTC              SatoshiLite          265
                 aantonop             255
TRX              justinsuntron        236
BTC              IvanOnTech           196
ETH              BarrySilbert         142
                 IvanOnTech           129
BTC              FEhrsam              123
ETH              CryptoCobain         105
BTC              KennethBosak         104
                 VitalikButerin       100
                 girlgone_crypto       98
                 TimDraper             97
                 CryptoCobain          95
USDT             justinsuntron         92
dtype: int64

In [77]:
df_path = LOCAL_PROCESSED_DATA_PATH / 'tweets_with_mentioned_coins.json'
df.reset_index(drop=True).to_json(df_path, orient='index')